#### Code for introducing Voltage Dependent Theta Neuron Model into Low Rank Connectivity SNNs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
class LowRankSNN:
    def __init__(self,N_E=1000,N_I=200,dt=0.01,T=30,RS=10,IS=100,taud_E=2,taud_I=5) -> None:
        pass

    def show(self):
        pass

    def add_lowrank(self):
        pass

    def remove_lowrank(self):
        pass

    def simulate(self,Input):
        pass



In [ ]:
# Functions


In [ ]:
# Initialiazation


In [ ]:
# Simulation

In [ ]:
# Draw the results

In [1]:
print({'a':1,'b':2})

{'a': 1, 'b': 2}
